In [ ]:
!pip install boto3

     |████████████████████████████████| 132 kB 15.5 MB/s 
     |████████████████████████████████| 8.7 MB 32.2 MB/s 
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 138 kB 65.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install cryptography

     |████████████████████████████████| 4.0 MB 12.6 MB/s 


**IMPORT REQUIRED LIBRARIES**

In [ ]:
import boto3
import base64
import logging
from cryptography.fernet import Fernet

In [ ]:
aws_access_key="{}"
aws_secret_key="{}"

**DEFINE ACCESS AND SECRET KEYS OF THE AWS ACCOUNT USED**

Enter the AWS access key id and secret access key from your respective AWS account

In [ ]:
import boto3
session = boto3.Session(
    aws_access_key_id="",
    aws_secret_access_key="",
    region_name=""
)

In [ ]:
kms_client=session.client('kms')

In [ ]:
from botocore.exceptions import ClientError

In [ ]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

**CREATING KMS KEY**

In [ ]:
def create_kms_key():
    """
    Creates a unique customer managed KMS key.
    """
    try:
        response = kms_client.create_key(Description='cmk-key',
                                         Tags=[{
                                             'TagKey': 'Name',
                                             'TagValue': 'hands-on-cloud-cmk'
                                         }])

    except ClientError:
        logger.exception('Could not create a CMK key.')
        raise
    else:
        return response


In [ ]:
kms = create_kms_key()

**META DATA OF THE KEY**

The meta data of the key gets displayed

In [ ]:
kms

**KEY ID GENERATED**

In [ ]:
key_id = kms['KeyMetadata']['KeyId']
key_id

'781fdbd0-4855-4086-8086-67db631f1729'

**CREATE DATA KEY**

In [ ]:
def create_data_key(cmk_id, key_spec='AES_256'):
    """Generate a data key to use when encrypting and decrypting data

    :param cmk_id: KMS CMK ID or ARN under which to generate and encrypt the
    data key.
    :param key_spec: Length of the data encryption key. Supported values:
        'AES_128': Generate a 128-bit symmetric key
        'AES_256': Generate a 256-bit symmetric key
    :return Tuple(EncryptedDataKey, PlaintextDataKey) where:
        EncryptedDataKey: Encrypted CiphertextBlob data key as binary string
        PlaintextDataKey: Plaintext base64-encoded data key as binary string
    :return Tuple(None, None) if error
    """
    try:
        response = kms_client.generate_data_key(KeyId=cmk_id, KeySpec=key_spec)
    except ClientError as e:
        logging.error(e)
        return None, None
    


    # Return the encrypted and plaintext data key
    return response['CiphertextBlob'], base64.b64encode(response['Plaintext']).decode(encoding="utf-8")

In [ ]:
ms=create_data_key(key_id, key_spec='AES_256')

In [ ]:
data_key=ms[1]
print(data_key)

uUr5OqqofP0vyUAZOyqA4S+p0bwU+xGgKKlHwSBZpO0=


**TESTING WITH A FILE**

**ENCRYPTION AND DECRYPTION**

In [ ]:
absolute_file_path_to_be_encrypted=input('Enter the filepath : ');
encrpted_file_name_with_path=input('Enter the output filepath : ');

f = Fernet(data_key)

with open(absolute_file_path_to_be_encrypted, 'rb') as original_file:
    original = original_file.read()

encrypted = f.encrypt(original)

with open (encrpted_file_name_with_path, 'wb') as encrypted_file:
    encrypted_file.write(encrypted)

Enter the filepath : input.txt
Enter the output filepath : output.txt


**ENCRYPTED FILE**

In [ ]:
with open(encrpted_file_name_with_path, 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)



**RECEIVED DATA AFTER DECRYPTION**

In [ ]:
with open('output.txt', 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

**Demo of Encryption and Decryption using AWS KMS is done!**